# `Heterogenous_graph`이종 그래프

In [1]:
pip install dgl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 24.7 MB/s eta 0:00:00
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


In [2]:
import dgl
import torch

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


이질적(heterogenous) 그래프의 경우, 서로 다른 관계에 놓인 노드들은 소스(source) 노드가 되거나 목적지(destination) 노드가 됩니다.   
`srcdata`와 `dstdata`는 (이름에서 source data, destination data가 보이죠?) 특히 이 두 타입의 노드에 저장되어 있습니다.  
이질적 그래프에 대한 더 자세한 내용을 알고 싶으시면, [DGL User Guide 1.5 Heterogeneous Graphs](https://docs.dgl.ai/guide/graph-heterogeneous.html#guide-graph-heterogeneous) 를 확인해 주세요.

![image.png](https://github.com/myeonghak/DGL-tutorial/blob/master/asset/user_guide_graphch_2.png?raw=1)

이질적 그래프는 다양한 타입의 노드와 엣지를 가질 수 있습니다.  
특정 타입의 노드/엣지는 각각의 독립적인 ID space와 피처 공간을 갖습니다.  
예를 들어, 위의 그림의 경우 user와 game 노드의 아이디는 각각 0부터 시작하고(독립적인 ID space), 각각 다른 피처를 갖습니다(독립적인 피처 공간).   


위의 예시에는 2개의 노드 타입(user, game)과 2개의 엣지 타입(follow, plays)이 있군요.  
유저끼리는 follow를 할것이고, 유저와 게임 사이에는 play라는 관계가 나오겠죠?  


In [3]:
# 3개의 노드 타입()과 3개의 엣지 타입을 가진 이질적 그래프를 생성합니다
# key 값의 3개 값 중 가운데는 "관계"를 나타내고, 양쪽은 source/destination node를 각각 나타냅니다.
# value 값의 부분에 있는 2 덩이의 torch.tensor는 각각 source/destination 노드의 피처를 의미합니다.

heterograph_data = {
   ('drug', 'interacts', 'drug'): (torch.tensor([0, 1]), torch.tensor([1, 2])),# 0과 1 연결, 1과 2 연결
   ('drug', 'interacts', 'gene'): (torch.tensor([0, 1]), torch.tensor([2, 3])),
   ('drug', 'treats', 'disease'): (torch.tensor([1]), torch.tensor([2]))
}
hetero_g = dgl.heterograph(heterograph_data)
print(hetero_g)

Graph(num_nodes={'disease': 3, 'drug': 3, 'gene': 4},
      num_edges={('drug', 'interacts', 'drug'): 2, ('drug', 'interacts', 'gene'): 2, ('drug', 'treats', 'disease'): 1},
      metagraph=[('drug', 'drug', 'interacts'), ('drug', 'gene', 'interacts'), ('drug', 'disease', 'treats')])


In [4]:
hetero_g

Graph(num_nodes={'disease': 3, 'drug': 3, 'gene': 4},
      num_edges={('drug', 'interacts', 'drug'): 2, ('drug', 'interacts', 'gene'): 2, ('drug', 'treats', 'disease'): 1},
      metagraph=[('drug', 'drug', 'interacts'), ('drug', 'gene', 'interacts'), ('drug', 'disease', 'treats')])

In [5]:
# 한 관계 'durg->interacts->gene'를 추출해 sub graph를 만듭니다.
sub_g = dgl.edge_type_subgraph(hetero_g, [('drug', 'interacts', 'gene')])
print(sub_g)

Graph(num_nodes={'drug': 3, 'gene': 4},
      num_edges={('drug', 'interacts', 'gene'): 2},
      metagraph=[('drug', 'gene', 'interacts')])


In [6]:
# 소스와 목적지 노드에 피처를 할당합니다. drug 노드와 gene 노드의 수가 다르다는 점에 주목해 주세요.

sub_g.srcdata['src_h'] = torch.randn(3,3) # (노드 수, feature 값 개수수)
sub_g.dstdata['dst_h'] = torch.randn(4,2)
print(sub_g)

Graph(num_nodes={'drug': 3, 'gene': 4},
      num_edges={('drug', 'interacts', 'gene'): 2},
      metagraph=[('drug', 'gene', 'interacts')])


In [7]:
sub_g.srcdata

{'src_h': tensor([[ 0.6883,  2.4891,  0.5295],
        [ 1.2424, -1.5627,  1.1472],
        [-0.6076,  1.6008,  1.0975]])}

In [8]:
sub_g.dstdata

{'dst_h': tensor([[-0.1696, -0.1181],
        [ 1.3223, -0.0417],
        [-0.1259, -0.6154],
        [-1.0470,  1.2929]])}

`srcdata`와 `dstdata`에 대한 더 많은 사용법은 5_message_passing과 large graph task 튜토리얼에서 확인하실 수 있습니다.